In [76]:
!pip install transformers --quiet
!pip install tokenizers --quiet
!pip install datasets --quiet
!pip install wget --quiet
!pip install transformers[torch]
!pip install pytorch_lightning
# !pip install git+https://github.com/PyTorchLightning/pytorch-lightning --quiet
!pip install datsets transformers[sentencepiece]
!pip install sentencepiece
import wget
import os
import numpy as np
import itertools
import json
import string
import re
import requests
from bs4 import BeautifulSoup
from urllib.parse import urlparse
import pandas as pd
from torch.optim import AdamW
from torch.utils.data import DataLoader
from transformers import AutoTokenizer, T5Tokenizer, T5ForConditionalGeneration
import torch

import pytorch_lightning as pl
from torchmetrics.text.rouge import ROUGEScore

ERROR: Could not find a version that satisfies the requirement datsets (from versions: none)
ERROR: No matching distribution found for datsets


In [77]:

ROBOTS = '/robots.txt'
Sitemaps = [
    '/sitemap.xml',
    '/sitemap-index.xml'
    '/sitemap.php'
    '/sitemap.txt'
    '/sitemap.xml.gz'
    '/sitemap/'
    '/sitemap/sitemap.xml'
    '/sitemapindex.xml'
    '/sitemap/index.xml'
    '/sitemap1.xml'
]
headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 14_1) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.1 Safari/605.1.15'
}

model_checkpoint = 't5-base'
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

MAX_LEN = 512
SUMMARY_MAX_LEN=128
TRAIN_BATCH_SIZE=2
VALID_BATCH_SIZE=2
LEARNING_RATE = 3e-4
BATCH_SIZE = 2
N_EPOCHS = 1

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5_fast.py:160: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to enc

Helper Functions

In [78]:
def getInputOutputNotTokenizedForSummary(xml, list_product_per_xml):
    input = []
    tags = []
    # huge XML
    xml_string = xml[0]
    lines = xml_string.split('\n')
    last_index = 0
    product_len = len(list_product_per_xml) - 1
    textCumumated = ""
    for i, text in enumerate(lines):
        if last_index > product_len:
            break
        product = list_product_per_xml[last_index]
        # this BeautifulSoup is there just to handle HTML entities because these can happen and the texts won't match
        text1 = BeautifulSoup(text, "xml")
        if product in text or product in text1.getText():
            last_index += 1
            # input.append(textToAdd)
            textCumumated += text
            input.append(textCumumated)
            tags.append(product)
            textCumumated = ""
        else:
            textCumumated +=  str(text)

    return input, tags



def getTextAndProductsForSite(url):
    try:
        requests.get(url, headers=headers)
    except:
        print("site" + url + "is down")
        return
    urlRobots = url + ROBOTS
    r = requests.get(urlRobots, headers=headers)
    m2 = BeautifulSoup(r.content, "html.parser")
    listProp = m2.text.split('\n')
    sitemap = extractSiteMapfromRobotsTxt(listProp)
    if '' == sitemap:
        sitemap = tryKnownSitepath(url)
        if '' == sitemap:
            print("could not get data for site :" + url)
            # break for loop since we dont know the sitepath
            return

    r1 = requests.get(sitemap, headers=headers)
    sitemapXml = BeautifulSoup(r1.content, "xml", from_encoding='utf-8')
    # in sitemap we have links we go in one by one,  some of these will contain the products
    listpaths = sitemapXml.findAll('loc')
    stringsPage = []
    products_page = []
    dictList = []
    for mainpath in listpaths:
        r = requests.get(mainpath.text, headers=headers)
        responsemainPage = BeautifulSoup(r.content, "xml")
        x1 = responsemainPage.findAll("title")
        # string_page = [responsemainPage.getText()]
        string_page = [str(responsemainPage)]
        product_page = [str(x.getText()) for x in x1]
        stringsPage.append(string_page)
        products_page.append(product_page)
        # dict ={
        #     "string_pageX": string_page,
        #     "products_pageX": product_page
        # }
        # dictList.append(dict)

    if len(stringsPage) == 0 or len(products_page) == 0:
        return None
    return {
        "string_page": stringsPage,
        "products_page": products_page,
        # "dict":dictList
    }


def extractSiteMapfromRobotsTxt(listProp):
    sitemap = ''
    for line in listProp:
        try:
            l1 = line.split(' ')
            if l1[0] == 'Sitemap:':
                sitemap = l1[1]
        except:
            pass
    if sitemap == "":
        # use sitemap list
        pass
    return sitemap


def getBasepage(link):
    parsed = urlparse(link)
    base = parsed.netloc
    scheme = parsed.scheme
    page = scheme + '://' + base
    return page


def tryKnownSitepath(url):
    for sitePath in Sitemaps:
        urltried = url + sitePath
        r1 = requests.get(urltried)
        if (r1.status_code == 200):
            return urltried
    return ""

Load Data

In [79]:
#download site Data only if not provided already
data_already_downloaded = True
if not data_already_downloaded:
    df = pd.read_csv('./data/furniture stores pages.csv')
    df = df.dropna()
    df_base_URL = df['max(page)'].map(getBasepage)
    df_base_URL_np = df_base_URL.values
    datasetList = []
    # download site data , XML and list of product for each site and put in list
    for i in range(100):
        url = df_base_URL_np[i]
        try:
            siteData = getTextAndProductsForSite(url)
            datasetList.append(siteData)
        except:
            continue
    # remove None elements which correspond to sites that give connection exception
    datasetListCleanNone = [i for i in datasetList if i is not None]
    # save data as JSON
    with open("datasetRaw1.json", "w") as outfile:
        json.dump(datasetListCleanNone, outfile)
#preprocess to Input Output form only if not already done or new data
data_already_preprocessed = True
if not data_already_preprocessed:
    with open('datasetRaw1.json') as user_file:
        parsed_json = json.load(user_file)
    inputs, tags = [], []
    # from the downloaded data pre process in Input Output form

    for siteDict in parsed_json:
        list_xmls = siteDict['string_page']
        list_product_per_xml = siteDict['products_page']
        lenght = len(list_xmls)

        for i in range(lenght):
            current_xml = list_xmls[i]
            current_product_found = list_product_per_xml[i]
            input, tag = getInputOutputNotTokenizedForSummary(current_xml, current_product_found)
            inputs.extend(input)
            tags.extend(tag)
    # save data to CVS
    trainDF = pd.DataFrame({'text': inputs, 'summary': tags})
    trainDF.to_csv('data100SitesNEW_Summary.csv', index=False, encoding='utf-8')

driveLink = "https://drive.google.com/uc?export=download&id=1iltGmHYooAe5pTpeF7LZyh8FzJlg9fjl"
data_dir = './data'

dataAlreadyExists = os.path.exists(data_dir)
if not dataAlreadyExists:
  os.makedirs(data_dir)
  file_name = wget.download(driveLink,out = data_dir)
# Data is ready here
# trainDF = pd.read_csv('data100SitesNEW_trunc.csv', encoding='utf-8')
trainDF = pd.read_csv('./data/data100SitesNEW_Summary.csv', encoding='utf-8')
# we use literal_eval because when the CVS is saved list of strings becomes String in each entry


trainDF.text = 'summarize: ' + trainDF.text
train_size = 0.9
train_dataset=trainDF.sample(frac=train_size)
val_dataset=trainDF.drop(train_dataset.index).reset_index(drop=True)
train_dataset = train_dataset.reset_index(drop=True)
tokenizer = T5Tokenizer.from_pretrained("t5-base")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5.py:240: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/p

Dataset

In [80]:
class SummaryDataset(torch.utils.data.Dataset):


    def __init__(self, data: pd.DataFrame, tokenizer: T5Tokenizer, source_len=512, summ_len=128):
        self.tokenizer = tokenizer
        self.data = data
        self.source_len = source_len
        self.summ_len = summ_len
        self.summary = self.data.summary
        self.text = self.data.text

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index:int):
        text = str(self.text[index])
        text = ' '.join(text.split())

        summary = str(self.summary[index])
        summary = ' '.join(summary.split())

        source = self.tokenizer.batch_encode_plus([text], max_length=self.source_len, pad_to_max_length=True,
                                                  return_tensors='pt')
        target = self.tokenizer.batch_encode_plus([summary], max_length=self.summ_len, pad_to_max_length=True,
                                                  return_tensors='pt')
        target['input_ids'][target['input_ids']==0] =-100
        source_ids = source['input_ids'].flatten()
        source_mask = source['attention_mask'].flatten()
        target_ids = target['input_ids'].flatten()

        return {
            'input_ids': source_ids.to(dtype=torch.long),
            'attention_mask': source_mask.to(dtype=torch.long),
            'labels': target_ids.to(dtype=torch.long),
        }

print(val_dataset.head())
print(train_dataset.head())


                                                text  \
0  summarize: <image:caption/></image:image></url...   
1  summarize: <image:caption/></image:image></url...   
2  summarize: <image:caption/></image:image></url...   
3  summarize: <image:caption/></image:image></url...   
4  summarize: <image:caption/></image:image></url...   

                                             summary  
0  Fire Pit BBQ Charcoal Grill Smoker Portable Ou...  
1  Greenhouse Garden Shed Tunnel Plant Green Hous...  
2  Rustic Wall Shelves Display Bookshelf Industri...  
3         Argo PU Leather Reclining Armchair - Black  
4  Kids Ride On Bulldozer Digger Electric Car - Y...  
                                                text  \
0  summarize: <image:caption/></image:image></url...   
1  summarize: </image:image><image:image><image:l...   
2  summarize: <image:caption/></image:image></url...   
3  summarize: <image:caption/></image:image></url...   
4  summarize: <image:caption/></image:image></url... 

PyTorch-Lightning Data loader

In [81]:
    class SummaryDataModule(pl.LightningDataModule):
        def __init__(
                self,
                train_df: pd.DataFrame,
                test_df: pd.DataFrame,
                tokenizer: T5Tokenizer,
                batch_size: int = 8,
                source_max_token_len: int = MAX_LEN,
                target_max_token_len: int = SUMMARY_MAX_LEN,
        ):
            super().__init__()
            self.train_df = train_df
            self.test_df = test_df
            self.tokenizer = tokenizer
            self.batch_size = batch_size
            self.source_max_token_len = source_max_token_len
            self.target_max_token_len = target_max_token_len

        def setup(self,stage=None):
            self.train_dataset = SummaryDataset(
                self.train_df,
                self.tokenizer,
                self.source_max_token_len,
                self.target_max_token_len
            )
            self.test_dataset = SummaryDataset(
                self.test_df,
                self.tokenizer,
                self.source_max_token_len,
                self.target_max_token_len
            )
            self.val_dataset = SummaryDataset(
                self.test_df,
                self.tokenizer,
                self.source_max_token_len,
                self.target_max_token_len
            )


        def train_dataloader(self):
            return DataLoader(
                self.train_dataset,
                batch_size=self.batch_size,
                shuffle=True,
                num_workers=4
            )

        def val_dataloader(self):
            return DataLoader(
                self.test_dataset,
                batch_size=self.batch_size,
                num_workers=4
            )

        def test_dataloader(self):
            return DataLoader(
                self.test_dataset,
                batch_size=1,
                num_workers=4
            )


    data_module = SummaryDataModule(train_dataset, val_dataset, tokenizer, batch_size=BATCH_SIZE)
    data_module.setup()


Model

In [82]:
    class SummmaryModel(pl.LightningModule):
        def __init__(self):
            super().__init__()
            self.model = T5ForConditionalGeneration.from_pretrained(model_checkpoint, return_dict=True)

        def forward(self, input_ids, attention_mask, labels=None):
            output = self.model(
                input_ids,
                attention_mask=attention_mask,
                labels=labels)
            return output.loss, output.logits

        def training_step(self, batch, batch_idx):
            input_ids = batch['input_ids']
            attention_mask = batch['attention_mask']
            labels = batch['labels']
            loss, outputs = self(input_ids, attention_mask, labels)
            self.log("train_loss", loss, prog_bar=True, logger=True)
            return {"loss": loss, "predictions": outputs, "labels": labels}

        def validation_step(self, batch, batch_idx):
            input_ids = batch['input_ids']
            attention_mask = batch['attention_mask']
            labels = batch['labels']
            loss, outputs = self(input_ids, attention_mask, labels)
            self.log("val_loss", loss, prog_bar=True, logger=True)
            return loss

        def test_step(self, batch, batch_idx):
            input_ids = batch['input_ids']
            attention_mask = batch['attention_mask']
            labels = batch['labels']
            loss, outputs = self(input_ids, attention_mask, labels)
            self.log("test_loss", loss, prog_bar=True, logger=True)
            return loss

        def configure_optimizers(self):
            optimizer = AdamW(self.parameters(), lr=LEARNING_RATE)
            return optimizer

Train

In [84]:
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger
import torchmetrics
model = SummmaryModel()
checkpoint_callback = ModelCheckpoint(
    dirpath="checkpoints",
    filename="best-checkpoint",
    save_top_k=1,
    verbose=True,
    monitor="val_loss",
    mode="min"
)
N_EPOCHS=2
logger = TensorBoardLogger("training-logs", name="Summary")
trainer = pl.Trainer(
    # logger = logger,
    callbacks=[checkpoint_callback],
    max_epochs=N_EPOCHS,
    enable_progress_bar=True
)
trainer.fit(model, data_module)
# trainer.test()
trained_model = SummmaryModel.load_from_checkpoint("checkpoints/best-checkpoint.ckpt")
trained_model.freeze()

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:639: Checkpoint directory /content/checkpoints exists and is not empty.
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 222 M 
-----------------------------------------------------
222 M     Trainable params
0         Non-trainable params
222 M     Total params
891.614   Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:Epoch 1, global step 98068: 'val_loss' reached 0.03883 (best 0.03883), saving model to '/content/checkpoints/best-checkpoint-v1.ckpt' as top 1
INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=2` reached.


Evaluate with RougeL

In [85]:
def generate_summary(seed_line, model_, num_beam=4, penalty_length=0.2):

    # Put the model on eval mode
    model_.eval()
    encoded_query = tokenizer(seed_line,
                              return_tensors='pt', pad_to_max_length=True, truncation=True, max_length=512)
    input_ids = encoded_query["input_ids"]
    attention_mask = encoded_query["attention_mask"]
    generated_answer = model.model.generate(input_ids, attention_mask=attention_mask,
                                      max_length=128, top_p=0.95, top_k=50)
    decoded_answer = tokenizer.decode(generated_answer[0])

    return decoded_answer

rouge = torchmetrics.text.rouge.ROUGEScore()
test100 = val_dataset.head(10)
test100.iloc(0)
rougeL_fmeasure = []
for text  ,summary in test100.values:
    summaryModel = generate_summary(text,model)
    print("----text----" +"\n" + text)
    print("----summary----" +"\n" +summary)
    print("----summaryModel----" +"\n" +summaryModel)
    result = rouge(summaryModel, summary)
    rougeL_fmeasure.append(result["rougeL_fmeasure"])



/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2614: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


----text----
summarize: <image:caption/></image:image></url><url><loc>https://www.factorybuys.com.au/products/30-inch-portable-fire-pit-free-shipping</loc><lastmod>2023-12-08T02:02:38+10:00</lastmod><changefreq>daily</changefreq><image:image><image:loc>https://cdn.shopify.com/s/files/1/1293/8737/products/FPIT-UFO-7676-99.jpg?v=1657623092</image:loc><image:title>Fire Pit BBQ Charcoal Grill Smoker Portable Outdoor Camping Garden Pits 30"</image:title>
----summary----
Fire Pit BBQ Charcoal Grill Smoker Portable Outdoor Camping Garden Pits 30"
----summaryModel----
<pad> Fire Pit BBQ Charcoal Grill Smoker Portable Outdoor Camping Garden Pits 30"</s>
----text----
summarize: <image:caption/></image:image></url><url><loc>https://www.factorybuys.com.au/products/4-shelf-greenhouse-with-cover</loc><lastmod>2023-12-08T02:02:38+10:00</lastmod><changefreq>daily</changefreq><image:image><image:loc>https://cdn.shopify.com/s/files/1/1293/8737/products/GH-MINI-4T-TP-00.jpg?v=1608193835</image:loc><image

In [86]:
arr = np.array(rougeL_fmeasure )
print(len(arr))
print(np.mean(arr))

10
0.89126146


In [88]:
rouge = torchmetrics.text.rouge.ROUGEScore()
test100 = val_dataset.head(1000)
rougeL_fmeasure = []
for text  ,summary in test100.values:
    summaryModel = generate_summary(text,model)
    result = rouge(summaryModel, summary)
    rougeL_fmeasure.append(result["rougeL_fmeasure"])


arr = np.array(rougeL_fmeasure )
print(len(arr))
print(np.mean(arr))

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2614: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


1000
0.8929364


test on a new site

In [89]:
pageTest ='https://vauntdesign.com/'
siteData = getTextAndProductsForSite(pageTest)


paragraph = str(BeautifulSoup(siteData['string_page'][0][0], "xml"))
paragraph = paragraph[500:1024]
print("-------paragraph:-------"+"\n" + paragraph)

summaryModel = generate_summary(paragraph,model)
print("-------summary:-------"+ "\n"+ summaryModel)


-------paragraph:-------
mage:title>Firms Bedside Table</image:title>
<image:caption/>
</image:image>
</url>
<url>
<loc>https://vauntdesign.com/products/bayon-salmon-pink-scatter-cushion</loc>
<lastmod>2024-01-07T21:42:48+00:00</lastmod>
<changefreq>daily</changefreq>
<image:image>
<image:loc>https://cdn.shopify.com/s/files/1/2001/3223/products/Salmonpinkcushion.jpg?v=1610199728</image:loc>
<image:title>Bayon Salmon Pink Scatter Cushion</image:title>
<image:caption>Salmon pink scatter cushion</image:caption>
</image:image>
</url>
<url>
<loc>ht
-------summary:-------
<pad> Bayon Salmon Pink Scatter Cushion</s>


In [90]:
from google.colab import drive
drive.mount('/content/drive')

!cp -r /content/checkpoints /content/drive/MyDrive/InterviuNer/T5Summary

# trainer.test()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
